**Updates**

So even with this new format, there seems to be some issue with the dask implementation! 

This time its with the distributed nature of dask. Workers time out and data is lost. 

Debugging this is beyond the scope of this project as far as I'm concerned especially consdering that I have spent quite a bit of time trying to get this to work.

The good thing with the new data format is that I can easily process it sequentially. So I split the files into a format that can fit on memory. So 17 files of approx 1G each (equivalent to the 17G single file).

I can then run pandas sequentially on this data. The pandas solution is actually quite fast.

And if I am still looking for a bit of parallelism I can do a batch script that creates a job for each file and ships it to the cluster.

I lose the fancy graphs that dask has, but I'll still do it quite fast.

In [1]:
import numpy as np
import json
import os
import pandas as pd

In [2]:
from collections import OrderedDict

In [3]:
symptom_db_json = os.path.join("/home/oagba/bulk/data/kk/json", "symptom_db.json")
condition_db_json = os.path.join("/home/oagba/bulk/data/kk/json", "condition_db.json")

In [4]:
with open(symptom_db_json) as fp:
    symptom_db = json.load(fp)
with open(condition_db_json) as fp:
    condition_db = json.load(fp)

In [5]:
symptom_vector = sorted(list(symptom_db.keys()))
condition_codes = sorted(list(condition_db.keys()))
condition_labels = {code: idx for idx, code in enumerate(condition_codes)}

In [6]:
from glob import glob

In [13]:
def _race_txform(val):
    race_code = {'white': 0, 'black':1, 'asian':2, 'native':3, 'other':4}
    return race_code.get(val)
def _label_txform(val, labels):
    return labels.get(val)
def _symptom_transform(val, labels):
    if type(val) is not str:
        print(val)
    parts = val.split(";")
    res = sum([labels.get(item) for item in parts])
    return res
def handle_bit_wise(val, comp):
    if val & comp > 0:
        return 1
    else:
        return 0

In [12]:
symptom_index_map = OrderedDict({code: 2**idx for idx, code in enumerate(symptom_vector)})

In [7]:
symptoms_csvs = "/shares/bulk/oagba/data/output_new_100k/symptoms/csv/split/x*"

In [15]:
output_dir = "/home/oagba/bulk/data/output_new_100k/parsed"

In [8]:
symptom_files = sorted(glob(symptoms_csvs))

In [21]:
def parse_data(filepath, condition_labels, symptom_map, output_dir):
    symptom_columns = ['PATIENT', 'GENDER', 'RACE', 'ETHNICITY', 'AGE_BEGIN', 'AGE_END',
       'PATHOLOGY', 'NUM_SYMPTOMS', 'SYMPTOMS']
    print("Parsing: %s" % filepath)
    filename = filepath.split("/")[-1]
    if filename == 'xaa':
        symptoms_df = pd.read_csv(filepath)
    else:
        symptoms_df = pd.read_csv(filepath, names=symptom_columns)
    # drop the guys that have no symptoms
    symptoms_df = symptoms_df.loc[symptoms_df.NUM_SYMPTOMS > 0]
    
    symptoms_df['LABEL'] = symptoms_df.PATHOLOGY.apply(_label_txform, labels=condition_labels)
    symptoms_df.RACE = symptoms_df.RACE.apply(_race_txform)
    symptoms_df.GENDER = symptoms_df.GENDER.apply(lambda gender: 0 if gender == 'F' else 1)
    symptoms_df = symptoms_df.rename(columns={'AGE_BEGIN': 'AGE'})
    symptoms_df['NSYMPTOMS'] = symptoms_df.SYMPTOMS.apply(_symptom_transform, labels=symptom_map)
    
    vector = sorted(symptom_map.keys())
    for idx, code in enumerate(vector):
        val = 2**idx
        symptoms_df[code] = (symptoms_df.NSYMPTOMS & 2**idx).gt(0).astype(np.uint8)
    
    ordered_keys = ['LABEL', 'GENDER', 'RACE', 'AGE'] + vector
    output_file = os.path.join(output_dir, "%s.csv" % filename)
    symptoms_df.to_csv(output_file)
    del symptoms_df
    print("Done parsing: %s" % filepath)

In [ ]:
for file in symptom_files:
    parse_data(file, condition_labels, symptom_index_map, output_dir)